In [64]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('ggplot')

In [65]:
df=pd.read_csv('/Users/chandan/Desktop/MyLA Data Project/MyLA_event_log_Fall2019-ANONYMIZED.csv')
df.head()

,id,timestamp,action,extra,username,course_id
0,8699,11/15/19 5:23,VIEW_GRADE_DISTRIBUTION,"{""course_id"": ""course_000000"", ""show_number_on...",subject_001244,course_000000
1,8892,11/18/19 15:51,VIEW_GRADE_DISTRIBUTION,"{""course_id"": ""course_000000"", ""show_number_on...",subject_001244,course_000000
2,8682,11/15/19 5:09,VIEW_GRADE_DISTRIBUTION,"{""course_id"": ""course_000000"", ""show_number_on...",subject_001228,course_000000
3,6716,10/16/19 14:01,VIEW_RESOURCE_ACCESS,"{""week_num_start"": 4, ""week_num_end"": 6, ""grad...",subject_000911,course_000000
4,6717,10/16/19 14:02,VIEW_ASSIGNMENT_PLANNING,"{""course_id"": ""course_000000"", ""percent_select...",subject_000911,course_000000


In [66]:
stud = pd.read_csv('/Users/chandan/Desktop/MyLA Data Project/MyLA_Student_Data_Fall2019_withCanvasCourses-ANONYMIZED copy 2.csv')
stud.head()

,Campus ID,Sex,International or Domestic,Acad Level BOT,Cum GPA,Math Plcmt,CanvasCourseID_long
0,subject_001637,Male,International,Grad Mastr,3.673,NaN,course_000003
1,subject_001146,Female,Domestic,Grad Mastr,3.731,NaN,course_000003
2,subject_001546,Male,Domestic,Grad Mastr,3.877,25.0,course_000003
3,subject_001376,Female,Domestic,Grad Mastr,3.879,17.0,course_000003
4,subject_000932,Male,Domestic,Grad Mastr,3.770,12.0,course_000003


In [67]:
TOT_STUD_LS = list(set(stud['Campus ID'].to_list())) 

In [68]:
users_df = df.loc[df['username'].isin(TOT_STUD_LS) ]
users_df.username.nunique()

421

In [69]:
## students in log sheet that are not in student database -- NEED TO BE CLARIFIED
non_users_df1 = df.loc[~ df['username'].isin(TOT_STUD_LS) ]
non_users_df1.username.nunique()

10

In [84]:
# Usernames in the log file that are not present in the student database
NON_USERS_LS1 =non_users_df1.username.unique()
NON_USERS_LS2 = [user for user in TOT_STUD_LS if user not in users_df.username.unique()]

NON_USERS_LS = list(set(list(NON_USERS_LS1) + list( NON_USERS_LS2) ))

In [71]:
tmp = users_df.groupby(['username','action']).count() ['id'].reset_index()
module_user_cnt_df = tmp.pivot(index='username',columns='action',values = 'id').fillna(0).reset_index()
print(module_user_cnt_df.shape)
module_user_cnt_df.head()

(421, 5)


action,username,VIEW_ASSIGNMENT_PLANNING,VIEW_GRADE_DISTRIBUTION,VIEW_RESOURCE_ACCESS,VIEW_SET_DEFAULT
0,subject_000115,0.0,1.0,0.0,2.0
1,subject_000179,2.0,5.0,1.0,8.0
2,subject_000339,0.0,1.0,0.0,0.0
3,subject_000363,0.0,2.0,1.0,0.0
4,subject_000398,4.0,6.0,2.0,8.0


In [72]:
module_user_cnt_df.rename(columns = {'VIEW_ASSIGNMENT_PLANNING':'VAP',
                                    'VIEW_GRADE_DISTRIBUTION' : 'VGD',
                                    'VIEW_RESOURCE_ACCESS' : 'VRA',
                                    'VIEW_SET_DEFAULT' : 'VSD'}, inplace= True)

In [73]:
# 20 10 8 4 <2
def get_category(df,COL_NAMES_LS):
    OP_LS = []
    my_df = df.copy()
    for COL_NAME in COL_NAMES_LS:
        TMP = pd.cut(x=my_df[COL_NAME], bins=[-1, 1, 4, 8, 10, 20,100],
                    labels=['NON USERS','LOW USERS' ,'MEDIUM USERS', 'HIGH USERS','SUPER USERS','SUPER HIGH USERS']).fillna('SUPER HIGH USERS').reset_index()
        op = TMP[COL_NAME].to_list()
        OP_LS.append(op)
#     print(OP_LS)
    TMP = pd.DataFrame(OP_LS ).T
    TMP.columns = [col+'_CATEGORY' for col in COL_NAMES_LS]
    RET_DF = pd.concat([my_df,TMP],axis=1)
    return RET_DF

In [74]:
MODULE_LS = ['VAP', 'VGD','VRA','VSD']

In [75]:
categorized_users = get_category(module_user_cnt_df,MODULE_LS)
categorized_users

,username,VAP,VGD,VRA,VSD,VAP_CATEGORY,VGD_CATEGORY,VRA_CATEGORY,VSD_CATEGORY
0,subject_000115,0.0,1.0,0.0,2.0,NON USERS,NON USERS,NON USERS,LOW USERS
1,subject_000179,2.0,5.0,1.0,8.0,LOW USERS,MEDIUM USERS,NON USERS,MEDIUM USERS
2,subject_000339,0.0,1.0,0.0,0.0,NON USERS,NON USERS,NON USERS,NON USERS
3,subject_000363,0.0,2.0,1.0,0.0,NON USERS,LOW USERS,NON USERS,NON USERS
4,subject_000398,4.0,6.0,2.0,8.0,LOW USERS,MEDIUM USERS,LOW USERS,MEDIUM USERS
...,...,...,...,...,...,...,...,...,...
416,subject_001750,0.0,2.0,0.0,2.0,NON USERS,LOW USERS,NON USERS,LOW USERS
417,subject_001754,0.0,3.0,0.0,0.0,NON USERS,LOW USERS,NON USERS,NON USERS
418,subject_001756,0.0,1.0,0.0,0.0,NON USERS,NON USERS,NON USERS,NON USERS
419,subject_001757,0.0,1.0,0.0,7.0,NON USERS,NON USERS,NON USERS,MEDIUM USERS


In [76]:
categorized_users.VAP_CATEGORY.value_counts()

NON USERS           318
LOW USERS            80
MEDIUM USERS         14
HIGH USERS            4
SUPER USERS           4
SUPER HIGH USERS      1
Name: VAP_CATEGORY, dtype: int64

In [77]:
code_dict = {'SUPER HIGH USERS':5,'SUPER USERS':4,'HIGH USERS': 3,'MEDIUM USERS':2,'LOW USERS': 1, 'NON USERS': 0}

In [78]:
for i in MODULE_LS:
    categorized_users[i+'_SCORE'] = categorized_users[i+'_CATEGORY'].map(code_dict)

In [79]:
categorized_users['TOTAL_SCORE'] = categorized_users.apply(lambda x: sum([x[i+'_SCORE'] for i in MODULE_LS]), axis=1)
categorized_users

,username,VAP,VGD,VRA,VSD,VAP_CATEGORY,VGD_CATEGORY,VRA_CATEGORY,VSD_CATEGORY,VAP_SCORE,VGD_SCORE,VRA_SCORE,VSD_SCORE,TOTAL_SCORE
0,subject_000115,0.0,1.0,0.0,2.0,NON USERS,NON USERS,NON USERS,LOW USERS,0,0,0,1,1
1,subject_000179,2.0,5.0,1.0,8.0,LOW USERS,MEDIUM USERS,NON USERS,MEDIUM USERS,1,2,0,2,5
2,subject_000339,0.0,1.0,0.0,0.0,NON USERS,NON USERS,NON USERS,NON USERS,0,0,0,0,0
3,subject_000363,0.0,2.0,1.0,0.0,NON USERS,LOW USERS,NON USERS,NON USERS,0,1,0,0,1
4,subject_000398,4.0,6.0,2.0,8.0,LOW USERS,MEDIUM USERS,LOW USERS,MEDIUM USERS,1,2,1,2,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
416,subject_001750,0.0,2.0,0.0,2.0,NON USERS,LOW USERS,NON USERS,LOW USERS,0,1,0,1,2
417,subject_001754,0.0,3.0,0.0,0.0,NON USERS,LOW USERS,NON USERS,NON USERS,0,1,0,0,1
418,subject_001756,0.0,1.0,0.0,0.0,NON USERS,NON USERS,NON USERS,NON USERS,0,0,0,0,0
419,subject_001757,0.0,1.0,0.0,7.0,NON USERS,NON USERS,NON USERS,MEDIUM USERS,0,0,0,2,2


In [80]:
categorized_users['VGD_SCORE']= categorized_users['VGD_SCORE']+categorized_users['VSD_SCORE']
categorized_users

,username,VAP,VGD,VRA,VSD,VAP_CATEGORY,VGD_CATEGORY,VRA_CATEGORY,VSD_CATEGORY,VAP_SCORE,VGD_SCORE,VRA_SCORE,VSD_SCORE,TOTAL_SCORE
0,subject_000115,0.0,1.0,0.0,2.0,NON USERS,NON USERS,NON USERS,LOW USERS,0,1,0,1,1
1,subject_000179,2.0,5.0,1.0,8.0,LOW USERS,MEDIUM USERS,NON USERS,MEDIUM USERS,1,4,0,2,5
2,subject_000339,0.0,1.0,0.0,0.0,NON USERS,NON USERS,NON USERS,NON USERS,0,0,0,0,0
3,subject_000363,0.0,2.0,1.0,0.0,NON USERS,LOW USERS,NON USERS,NON USERS,0,1,0,0,1
4,subject_000398,4.0,6.0,2.0,8.0,LOW USERS,MEDIUM USERS,LOW USERS,MEDIUM USERS,1,4,1,2,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
416,subject_001750,0.0,2.0,0.0,2.0,NON USERS,LOW USERS,NON USERS,LOW USERS,0,2,0,1,2
417,subject_001754,0.0,3.0,0.0,0.0,NON USERS,LOW USERS,NON USERS,NON USERS,0,1,0,0,1
418,subject_001756,0.0,1.0,0.0,0.0,NON USERS,NON USERS,NON USERS,NON USERS,0,0,0,0,0
419,subject_001757,0.0,1.0,0.0,7.0,NON USERS,NON USERS,NON USERS,MEDIUM USERS,0,2,0,2,2


In [81]:
del categorized_users['VSD_SCORE']
categorized_users

,username,VAP,VGD,VRA,VSD,VAP_CATEGORY,VGD_CATEGORY,VRA_CATEGORY,VSD_CATEGORY,VAP_SCORE,VGD_SCORE,VRA_SCORE,TOTAL_SCORE
0,subject_000115,0.0,1.0,0.0,2.0,NON USERS,NON USERS,NON USERS,LOW USERS,0,1,0,1
1,subject_000179,2.0,5.0,1.0,8.0,LOW USERS,MEDIUM USERS,NON USERS,MEDIUM USERS,1,4,0,5
2,subject_000339,0.0,1.0,0.0,0.0,NON USERS,NON USERS,NON USERS,NON USERS,0,0,0,0
3,subject_000363,0.0,2.0,1.0,0.0,NON USERS,LOW USERS,NON USERS,NON USERS,0,1,0,1
4,subject_000398,4.0,6.0,2.0,8.0,LOW USERS,MEDIUM USERS,LOW USERS,MEDIUM USERS,1,4,1,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...
416,subject_001750,0.0,2.0,0.0,2.0,NON USERS,LOW USERS,NON USERS,LOW USERS,0,2,0,2
417,subject_001754,0.0,3.0,0.0,0.0,NON USERS,LOW USERS,NON USERS,NON USERS,0,1,0,1
418,subject_001756,0.0,1.0,0.0,0.0,NON USERS,NON USERS,NON USERS,NON USERS,0,0,0,0
419,subject_001757,0.0,1.0,0.0,7.0,NON USERS,NON USERS,NON USERS,MEDIUM USERS,0,2,0,2


In [ ]:
Dict_overall_perf = {'SUPER HIGH USERS':5,'SUPER USERS':4,'HIGH USERS': 3,'MEDIUM USERS':2,'LOW USERS': 1, 'NON USERS': 0}